In [ ]:
!apt-get update -y
!apt-get install -y \
libglib2.0-0 \
libnss3 \
libdbus-glib-1-2 \
libgconf-2-4 \
libfontconfig1 \
libvulkan1 \
gconf2-common \
libwayland-server0 \
libgbm1 \
udev \
libu2f-udev 
!apt --fix-broken install -y  
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chrome-linux64.zip
!unzip /tmp/chrome-linux64.zip -d /usr/bin/
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver-linux64.zip -d /usr/bin/
!apt install -y python3-selenium
!pip install selenium==4.7.0
!pip install kafka-python webdriver_manager
!pip install -q gdown

import time, random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager
import pickle
from kafka import KafkaProducer
import json 
import os
from tqdm import tqdm

In [ ]:
!git clone https://github.com/imnotnta/bd.git
%cd /kaggle/working/bd
!git checkout Huy
%cd /kaggle/working/bd

In [ ]:
CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driver
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            "--incognito",
            "--window-size=1920x1080"
            "user-agent=Chrome/116.0.5845.96"
        ],
    }
    options = add_driver_options(driver_config["options"])
    options.binary_location = CHROME_BINARY_LOCATION
    driver = webdriver.Chrome(
        executable_path=CHROMEDRIVER_BINARY_LOCATION,
        options=options)
    return driver
driver = initialize_driver()

In [ ]:
username = '____'
password = '____'

In [ ]:
driver.get("https://twitter.com/")
time.sleep(3)
driver.find_element("xpath","//a[@href='/login']").click()
time.sleep(3)
driver.find_element("xpath","//label").click()
time.sleep(2)
driver.find_element("xpath","//label").send_keys(username)
time.sleep(2)
driver.find_element("xpath","//label").send_keys(Keys.ENTER)
time.sleep(2)
driver.find_elements("xpath",'//input')[1].send_keys(password)
driver.find_elements("xpath",'//input')[1].send_keys(Keys.ENTER)

print("Finish enter password, you have 2 minutes")
time.sleep(60 * 2)

In [ ]:
!gdown ________________________

In [ ]:
# from PIL import Image

In [ ]:
# driver.get(f"https://twitter.com/cryptoueen999") #These posts are protected
driver.save_screenshot(f'logged-in-1.png')
# Image.open('logged-in-1.png')

In [ ]:
key_path = f"/kaggle/working/bd/key_file.txt"
with open(key_path, 'r') as f:
    line = f.readlines()[0].strip().lower()
    print(f"Key = {line}")
import os
os.remove(key_path)
if line != 'n':
    driver.find_elements("xpath","//input")[0].send_keys(line)
    driver.find_elements("xpath","//input")[0].send_keys(Keys.ENTER)
print("Đã nhập key xong, chờ 5s để vào trang đăng nhập")
time.sleep(5)

In [ ]:
driver.save_screenshot(f'logged-in-2.png')

In [ ]:
if "Subscribe to unlock new features and if eligible, receive a share of ads revenue" in driver.page_source:
    language = "English"
elif "Đăng ký để mở khóa các tính năng mới và nếu đủ điều kiện, bạn sẽ được nhận một khoản chia sẻ doanh thu từ quảng cáo" in driver.page_source:
    language = "Vietnamese"
else:
    assert False, "Không phải Tiếng Anh hay Tiếng Việt"
print(language)

In [ ]:
class TwitterCrawler:
    def __init__(self, producer: KafkaProducer = None, hashtags: list = None, driver = driver):
        self.driver = driver
        self.hashtags = hashtags
        self.producer = producer
        self.groups = []
        self.save_file = f"./crawled_data" 
        os.makedirs(f"{self.save_file}/all_users", exist_ok=True)
        os.makedirs(f"{self.save_file}/groups", exist_ok=True)
        os.makedirs(f"{self.save_file}/user_info", exist_ok=True)
    
    def extract_to_dictionary(self,text):
        data_dict = {'replies': 0, 'reposts': 0, 'likes': 0, 'views': 0, 'bookmarks': 0}
        data = text.split(',')
        for item in data:
            item = item.strip()
            if 'repl' in item:
                data_dict['replies'] = int(item.split()[0])
            elif 'repost' in item:
                data_dict['reposts'] = int(item.split()[0])
            elif 'like' in item:
                data_dict['likes'] = int(item.split()[0])
            elif 'bookmark' in item:
                data_dict['bookmarks'] = int(item.split()[0])

            elif 'views' in item:
                data_dict['views'] = int(item.split()[0])

        return data_dict
    
    def retrieve_tweets(self, username):
        count = 0
        iter = 1
        tweet_dict = {}
        while iter < 6 and count < 20:
            tweets = self.driver.find_elements("xpath","//article[@tabindex='0']")
            for i in range(len(tweets)):
                tweet_time = tweets[i].find_element("xpath",".//time[@datetime and @datetime!='none']")
                tweet_time = tweet_time.get_attribute('datetime') + ''
                if tweet_time in tweet_dict.keys():
                    continue
                else:
                    if tweets[i].find_elements("xpath",f'.//a[@href="/{username}" and contains(@dir,"ltr")]') != []:
                        repost = "True" 
                    else:
                        repost = "False"
                    try:    
                        interacts = tweets[i].find_elements("xpath",f'.//div[@role="group"]')[0].get_attribute('aria-label')

                        interacts = self.extract_to_dictionary(interacts)
                    except:
                        interacts = {'replies': 0, 'reposts': 0, 'likes': 0, 'views': 0, 'bookmarks': 0}
                    try:
                        langs = tweets[i].find_element("xpath", f'.//div[@lang and @lang!="none"]')
                        try:
                            tweet_content = ' '.join([line.text for line in langs.find_elements("xpath",f".//*")])
                        except:
                            tweet_content = 'Video'
                    except:
                        langs = "NA"
                        tweet_content = 'Video'

                    tweet_dict.update({tweet_time: {'repost': repost,'content': tweet_content, 'language': langs.get_attribute('lang') if langs != "NA" else "NA",
                                                    'interacts': interacts}})
                    count += 1
                    if count == 20:
                        break
            #Scroll down
            self.driver.execute_script(f"window.scrollTo(0,{3000*iter})")
            time.sleep(1)
            iter+= 1
        
        return tweet_dict
    
    def retrieve_basic_user_info(self,username):
        self.driver.get(f'https://twitter.com/{username}')
        time.sleep(2)
        
        page_src = self.driver.page_source
        if language == 'Vietnamese':
            if "Đã xảy ra lỗi. Thử tải lại" in page_src:
                return "Something went wrong. Try reloading."
#             elif "Các bài đăng này được bảo vệ" in page_src:
#                 return "These posts are protected"
        elif language == 'English':
            if "Something went wrong. Try reloading." in page_src:
                return "Something went wrong. Try reloading."
#             elif "These posts are protected" in page_src:
#                 return "These posts are protected"
    
        user_info = self.driver.find_elements("xpath",f'//script[@type="application/ld+json"]')
        jsontext = json.loads(user_info[0].get_attribute('innerHTML'))
        try:
            job = self.driver.find_element("xpath", "//span[@data-testid ='UserProfessionalCategory']/*[2]/*").text
        except:
            job = "NA"
            
        verified = self.driver.find_elements("xpath", '//div[@aria-label ="Provides details about verified accounts."]/*/*/*')
        if len(verified) == 0:
            type_account = 'normal'
        elif len(verified) == 1:
            if verified[0].get_attribute('clip-rule') == 'evenodd':
                type_account = 'government'
            else:
                type_account = 'verified'
        else:
            type_account = 'company'
        if len(self.driver.find_elements("xpath",'//div[@aria-label="Provides details about protected accounts."]')) > 0:
            tweets = {}
            protected = 'True' 
        else:
            protected = 'False'
            if len(self.driver.find_elements("xpath",'//div[@data-testid="emptyState"]')) > 0:
                tweets = {}
            else: 
                tweets = self.retrieve_tweets(username)
                

        return_dict = {'type':jsontext['author']['@type'],
                        'name':jsontext['author']['givenName'],
                    'location':jsontext['author']['homeLocation']['name'],
                    'description':jsontext['author']['description'],
                    'date_created': jsontext['dateCreated'],
                    'num_followers': jsontext['author']['interactionStatistic'][0]['userInteractionCount'],
                    'num_following': jsontext['author']['interactionStatistic'][1]['userInteractionCount'],
                    'num_tweets': jsontext['author']['interactionStatistic'][2]['userInteractionCount'],
                    'job': job, 'type_account': type_account, 'protected': protected, 'all_tweets': tweets}
        return return_dict
    
    def load_username(self, keyword):
        try:
            with open(f"{self.save_file}/all_users/{keyword}.pkl", "rb") as f:
                list_users = pickle.load(f)
            with open(f"{self.save_file}/user_info/info.pkl", "rb") as f:
                dct = pickle.load(f)
            users_left = list(set(list_users) - set(dct['Finished']) - set(dct['Error']))
            return users_left
        except:
            return []
        
    def load_crawled_username(self, keyword):
        return [x.split(".")[0] for x in os.listdir(f"{self.save_file}/user_info/{keyword}")]

    def write_user_info_to_file(self, keyword, user_info, user_name):
        with open(f"{self.save_file}/user_info/{keyword}/{user_name}.pkl", "wb") as f:
            pickle.dump(user_info, f)
            
    def write_user_info_to_kafka(self, keyword, user_info, user_name):
        self.producer.send(f"{keyword}_{user_name}", pickle.dumps(user_info))
    
    def crawl_all_users(self, hashtag, reverse=False):
        user_names = self.load_username(hashtag)
        os.makedirs(f"{self.save_file}/user_info/{hashtag}", exist_ok=True)
        file_log_he = open("/kaggle/working/logs.txt", 'w')
        crawled_users = self.load_crawled_username(hashtag)
        user_names = sorted(list(set(user_names) - set(crawled_users))) #Make sure that we don't crawl one file we have already crawled
        print(f"Num user left: {len(user_names)}")

        assert reverse in [False, True]
        start, end, step = (len(user_names) - 1, -1, -1) if reverse else (0, len(user_names), 1)

        for idx in range(start, end, step):

            try:
                user_info = self.retrieve_basic_user_info(user_names[idx])
            except:
                file_log_he.write(f"Error when crawling user: {user_names[idx]}\n")
                print("Error when crawling user: ", user_names[idx])
                time.sleep(random.randint(20, 40))
                continue
            
            if isinstance(user_info, str):
                print('Skip crawling user: ', user_names[idx])
                file_log_he.write(f"Skip crawling user: {user_names[idx]}\n")
            else:
                self.write_user_info_to_file(hashtag, user_info, user_names[idx])
                # self.write_user_info_to_kafka(hashtag, user_info, user_names[idx])
                print('Finish crawling user: ', user_names[idx])
                file_log_he.write(f"Finish crawling user: {user_names[idx]}\n")
                
            time.sleep(random.randint(20, 40))

        print('Finish crawling users from hashtag: ', hashtag)

In [ ]:
# producer = KafkaProducer(bootstrap_servers=['34.142.194.212:29092'])
producer = None
crawler = TwitterCrawler(producer, ['Bitcoin'], driver = driver)
crawler.crawl_all_users('Bitcoin', reverse=False)